In [2]:
#necessary data manipulation libraries (standard imports)
#packages for data collection and manipulation
import pandas as pd
import numpy as np
import time, datetime
import ast
import warnings
import requests
#bokeh imports (visualization) libraries
from bokeh.io import curdoc
from bokeh.models.widgets import AutocompleteInput
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource, Select, DataTable, TableColumn, BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter, ColorBar, FactorRange, CustomJS, DatePicker
from bokeh.layouts import column, row
from bokeh.plotting import figure, Figure, show
#libraries necessary for natural language processing
import texthero as hero
from collections import defaultdict
from gensim import corpora, models, similarities
#coingecko API import
from pycoingecko import CoinGeckoAPI
warnings.filterwarnings('ignore')
#initialize the API
cg = CoinGeckoAPI()

# Read in csv files for dashboard data

In [3]:
"""
read in csv file and clean/organize csv file data for dashboard visualizations
This dataframe supplies close to all the data for the dashboard
The functions/visualizations filter down and manipulate this main dataframe

"""
total_market = pd.read_csv('/Users/thesavage/Dashboard_folder/Data/top_200.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
total_market['date'] = pd.to_datetime(total_market['date'])
total_market['tags'] = total_market['tags'].map(lambda x: ast.literal_eval(x)) #turn tags column into a list and not a string
total_market['daily returns'] = total_market.sort_values('date').groupby('coin_id')['prices'].transform(lambda x: (x/x.shift(1)-1)) #calculate daily returns for all coins
total_market = total_market.drop_duplicates(keep = 'first', subset = ['date', 'coin_id']).dropna()
exploded_market = total_market.explode('tags') #melt/explode the df for the tags column


In [4]:
"""
read in coin desciptions csv file. this df will be used for NLP and similarity queries
This dataframe/data will be exclusively used for the function that compares coin description semantics.
"""
descriptions = pd.read_csv('/Users/thesavage/Dashboard_folder/Data/descriptions.csv')
descriptions = descriptions.drop('Unnamed: 0', axis = 1).dropna().set_index('coin_id')


# Defining functions for manipulation and visualization, and widgets for user interactions

In [5]:
'''
For the dashboard we want to be able to pick a coin and gain information on that specific coin.
-This function will take in a given coin id and filter down the main dataframe to just a dataframe for the
specific chosen coin.
-We will use this dataframe as a data source for future graphs, and to derive other dataframe sources
'''
def get_coin_data(id_):
    market_df = total_market.set_index('date')
    df = market_df[market_df['coin_id'] == id_]
    df = df[['coin_id', 'prices', 'volume', 'mktcap', 'daily returns']].sort_values('date')
    return df.dropna()


In [6]:
#Widgets used for dashboard altering
#examples include the datepickers and the scatterplot tags dropdown menu and coin id input
coin_ids = list(total_market['coin_id'].unique())
coin_tags = list(exploded_market['tags'].unique())
coin_id = AutocompleteInput(title = 'Enter coin ID here', value = 'bitcoin', completions = coin_ids)
coin_tag = Select(title = 'Select a coin tag', value = 'Ethereum Ecosystem', options = coin_tags)
start_date = DatePicker(title='Select start date', value='2022-01-01', min_date = str(get_coin_data(coin_id.value).index.min()), max_date = str(get_coin_data(coin_id.value).index.max()))
end_date = DatePicker(title='Select end date', value='2022-08-08', min_date = str(get_coin_data(coin_id.value).index.min()), max_date = str(get_coin_data(coin_id.value).index.max()))
default_start = start_date.value
default_end = end_date.value



In [7]:
"""
This function will take in a scpecific coin id and create a dataframe
of coin semantic similarities, the dataframe is sorted from the highest
semantic similarity to the lowest.
"""
def nlp_df(id_):
    nlp_df = descriptions
    
    #create a corpus (collection of documents)
    corpus = nlp_df['description']
    #clean the corpus and replace the description column with the cleaned description
    clean_corpus = [doc.split() for doc in hero.clean(corpus)]
    #create a word frequency dictionary
    frequency = defaultdict(int)
    for text in clean_corpus:
        for token in text:
            frequency[token] += 1
    #process out the words that appear more than once (Tokenize)
    texts = [
        [token for token in text if frequency[token] > 2]
        for text in clean_corpus
    ]
    
    #Create a dictionary for distinct words in corpus
    dictionary = corpora.Dictionary(texts)
    #vectorize each documents using bag of words
    bow_vectorization = [dictionary.doc2bow(doc) for doc in texts]
    
    #initializing models and applying to corpus
    tfidf = models.TfidfModel(bow_vectorization)
    corpus_tfidf = tfidf[bow_vectorization]
    lsi = models.LsiModel(corpus_tfidf, id2word = dictionary, num_topics = 50)
    corpus_lsi = lsi[corpus_tfidf]
    
    #adding the vectorized transformations from models as columns in the dataframe
    nlp_df['description_cleaned'] = clean_corpus
    nlp_df['tokenized'] = texts
    nlp_df['bow_vectors'] = bow_vectorization
    nlp_df['tfidf'] = corpus_tfidf
    nlp_df['lsi'] = corpus_lsi
    
    #indexing and creating similarity queries (using lsi corpus)
    compare_vector = nlp_df['lsi'][id_]
    index = similarities.MatrixSimilarity(corpus_lsi)
    similarity_score = index[compare_vector]
    nlp_df['similarity_score'] = similarity_score
    
    similarity_df = nlp_df['similarity_score'].sort_values(ascending = False).to_frame().reset_index().drop(index = 0)
    
    return similarity_df
    

In [28]:
'''
This function takes in a specfic coin_id and daterange interval and returns
a dataframe used as a bokeh source for a correlation graph. The correlation is
a 5 day rolling correlation, that analyzes the specifieds coin correlation between the
returns of bitcoin and ethereum.

'''

def rolling_corr_df(id_, start = default_start, end = default_end):
    #Filter dataframe to the three coins (bitcoin, eth, and specified coin), and the daterange interval
    rolling_df = total_market[(total_market['coin_id'] == 'bitcoin') | (total_market['coin_id'] == 'ethereum') | (total_market['coin_id'] == id_)][['date', 'daily returns', 'coin_id']].sort_values('date').set_index('date')[start:end]
    #pivot results so the coin_ids have their own column and the values are the daily returns and do a 5 day rolling correlation
    rolling_df = rolling_df.pivot(columns = 'coin_id', values = 'daily returns').rolling(5).corr()
    #unstack
    rolling_df = rolling_df.rename(columns = {id_ : 'id_'})['id_'].unstack().rename(columns = {id_ : 'id_'})
    if id_ == 'bitcoin':
        rolling_df['bitcoin'] = 1
    elif id_ == 'ethereum':
        rolling_df['ethereum'] = 1
    else:
        rolling_df = rolling_df.reset_index()
    return rolling_df.drop_duplicates().dropna()


coin_id,date,coin_id,bitcoin,ethereum,tron
12,2022-01-05,bitcoin,1.000000,0.890780,0.892832
13,2022-01-05,ethereum,0.890780,1.000000,0.948105
14,2022-01-05,tron,0.892832,0.948105,1.000000
15,2022-01-06,bitcoin,1.000000,0.755362,0.958665
16,2022-01-06,ethereum,0.755362,1.000000,0.779661
...,...,...,...,...,...
655,2022-08-07,ethereum,0.930076,1.000000,0.355314
656,2022-08-07,tron,0.288446,0.355314,1.000000
657,2022-08-08,bitcoin,1.000000,0.917739,0.024845
658,2022-08-08,ethereum,0.917739,1.000000,0.193276


In [9]:
'''
This function sources our data for the dashboards scatter plot
It takes in a specific tag and filters down the dataframe to 
just the coins with that tag, and filters the datetime period to chosen interval
The resulting dataframe includes the cumulative return, the average daily return, 
and the standard deviation for each coin in the specified time interval.

'''
def scatter_plot_df(tag, start = default_start, end = default_end):
    tag_exclusive_df = exploded_market[(exploded_market['tags'] == tag)].set_index('date').sort_values(['date', 'coin_id'])[start:end]
    
    #calculates cumulative returns during current interval, if the interval is out of range, it calculates the max cumulative return
    cumulative_rets = []
    grouped = tag_exclusive_df.groupby('coin_id')
    #loop through each group(coin_id) in the grouped object
    for name, group in grouped:
         cumulative_rets.append(group['prices'].transform(lambda x: (x/x.shift(len(group.index.unique())-1)-1))[end])
        
    scatter_df = pd.DataFrame()
    scatter_df['average_returns'] = round(tag_exclusive_df[start: end].groupby('coin_id')['daily returns'].mean(), 4)
    scatter_df['std_returns'] = round(tag_exclusive_df[start: end].groupby('coin_id')['daily returns'].std(), 4)
    scatter_df['cumulative_return'] = cumulative_rets
    return scatter_df


In [10]:
#call function and create dataframe for coin typed into search bar widget
data_source = get_coin_data(coin_id.value)[default_start:default_end]
#create an interactive datasource for graphs (price, and stats table)
source = ColumnDataSource(data = data_source)

In [11]:
#create datasource for the similarity query datatable
similarity_source = nlp_df(coin_id.value)
similarity_source = ColumnDataSource(data = similarity_source)


In [27]:
#data and source for histogram (bokeh doesnt have a histogram method)
hist, edges = np.histogram(data_source['daily returns'], density=True, bins=50) #must use numpy histogram method to get buckets
hist_df = pd.DataFrame({'num' : hist, 'left' : edges[:-1], 'right' : edges[1:]})
hist_df['ret_interval'] = [(round(left,3), round(right,3)) for left, right in zip(hist_df['left'], hist_df['right'])]
hist_source = ColumnDataSource(data = hist_df)

In [28]:
#create datasource for rolling correlation graph
rolling_data = rolling_corr_df(coin_id.value, default_start, default_end)
rolling_source = ColumnDataSource(data = rolling_data)


In [29]:
#creating data source for scatter plot
scatter_df = scatter_plot_df(coin_tag.value)
scatter_source = ColumnDataSource(data = scatter_df)

In [30]:
#crafting data and setting data source for the descriptive statistics table
stats = data_source[['prices', 'daily returns']].describe().round(5).reset_index()
stats_source = ColumnDataSource(data = stats)


# Creating interactive visualizations and datatables

In [31]:
#create price-chart
price_chart = figure(title = (coin_id.value + " price").title(), height = 300, width = 700, x_axis_type = 'datetime')
price_chart.add_tools(HoverTool(tooltips=[("date",  "@date{%F}"),( "price", "$@{prices}{0.2f}" )],formatters={'@date': 'datetime', '@price': 'printf'}, mode = 'vline'))
price_chart.background_fill_color="#f5f5f5"
price_chart.grid.grid_line_color="white"
price_chart.xaxis.axis_label = 'Date'
price_chart.yaxis.axis_label = 'Price'
price_chart.axis.axis_line_color = None
price_chart.line(x = 'date', y = 'prices', line_width = 2, source = source)


GlyphRenderer(id='1353', ...)

In [32]:
#create histogram
histogram = figure(title = (coin_id.value + " histogram").title(), height = 600, width = 700)
histogram.add_tools(HoverTool(tooltips=[( "return_range",  "@ret_interval")]))
histogram.background_fill_color="#f5f5f5"
histogram.grid.grid_line_color="white"
histogram.xaxis.axis_label = 'Daily Returns'
histogram.yaxis.axis_label = 'Count'
histogram.axis.axis_line_color = None
histogram.quad(bottom = 0, top = 'num', left = 'left', right = 'right', fill_color = '#fdbb84', line_color = 'black', source = hist_source)


GlyphRenderer(id='1395', ...)

In [33]:
#create plot for rolling correlation between bitcoin and ethereum
rolling_plot = figure(title = (coin_id.value + " return correlation vs. bitcoin and ethereum").title(), height = 300, width = 700, x_axis_type = 'datetime')
rolling_plot.background_fill_color="#f5f5f5"
rolling_plot.grid.grid_line_color="white"
rolling_plot.xaxis.axis_label = 'Date'
rolling_plot.yaxis.axis_label = 'Return Correlation'
rolling_plot.axis.axis_line_color = None
plot1 = rolling_plot.line(x='date', y = 'bitcoin', line_width = 2, muted_alpha=0.2,legend_label = 'bitcoin', source = rolling_source)
plot2 = rolling_plot.line(x='date', y = 'ethereum', line_width = 2 , line_color = 'green', muted_alpha=0.2,legend_label = 'ethereum', source = rolling_source)

if coin_id.value == 'bitcoin':
    rolling_plot.add_tools(HoverTool(renderers = [plot2], tooltips=[( "date",  "@date{%F}"), ( "ethereum corr", "@{ethereum}{0.2f}" ), ( "bitcoin corr", "@{bitcoin}{0.2f}" )],formatters={'@date': 'datetime', '@ethereum corr': 'printf', "@bitcoin corr" : 'printf'}, mode = 'vline'))
else:
    rolling_plot.add_tools(HoverTool(renderers = [plot1], tooltips=[( "date",  "@date{%F}"), ( "ethereum corr", "@{ethereum}{0.2f}" ), ( "bitcoin corr", "@{bitcoin}{0.2f}" )],formatters={'@date': 'datetime', '@ethereum corr': 'printf', "@bitcoin corr" : 'printf'}, mode = 'vline'))

rolling_plot.legend.location = 'bottom_right'
rolling_plot.legend.click_policy="mute"



In [34]:
#create scatter plot
scatter_plot = figure(height = 750, width = 750)
scatter_plot.add_tools(HoverTool(tooltips=[("coin_id",  "@coin_id"), ( "average daily return", "@{average_returns}{0.4f}" ), ( "return standard deviation", "@{std_returns}{0.2f}" ), ("return over time interval", "@{cumulative_return}{0.4f}" )],formatters={'@coin_id': 'printf', '@return standard deviation': 'printf', "@average daily return" : 'printf', '@return over time interval' : 'printf'}))
scatter_plot.background_fill_color="#f5f5f5"
scatter_plot.grid.grid_line_color="white"
scatter_plot.xaxis.axis_label = 'Coins daily returns standard deviation'
scatter_plot.yaxis.axis_label = 'Coins average daily return'
scatter_plot.axis.axis_line_color = None
scatter_plot.scatter(x= 'std_returns', y = 'average_returns', source = scatter_source)


GlyphRenderer(id='1525', ...)

In [35]:
#create semantics similarity query datatable
columns = [
        TableColumn(field = "coin_id", title = "coin_id"),
        TableColumn(field = "similarity_score", title = "Similarity Score"),
    ]
similarity_table = DataTable(source = similarity_source, columns=columns, width=250, height=775)



In [36]:
#create describe datatable
stats_columns = [
                
                TableColumn(field = 'index', title = "Stats"),
                TableColumn(field = 'prices', title = "Prices"),
                TableColumn(field = 'daily returns', title = "Returns")
                 
                 ] 
stats_table = DataTable(source = stats_source, columns = stats_columns, width = 500, height = 200, index_position = None)

# Creating dashboard updates and callbacks

In [37]:
'''
The update_dash and update_scatter functions are used alongside (go hand and hand)
with the bokeh on_change method. The on_change method in the dashboards enviroment changes the value
of the dashboards' widgets. What the functions do is when a value of a widget is altered
the function updates the datasources for the plots and datatables with the new
selected values' data.
'''
def update_dash():
    id_ = coin_id.value
    coin_df = get_coin_data(id_)
    coin_df = coin_df[start_date.value : end_date.value]
    source.data = coin_df
    stats_source.data = round(coin_df[['prices', 'daily returns']].describe().reset_index(), 2)
    rolling_df = rolling_corr_df(id_, start = start_date.value, end = end_date.value)
    rolling_source.data = rolling_df
    hist, edges = np.histogram(coin_df['daily returns'], density=True, bins=50)
    hist_df = pd.DataFrame({'num' : hist, 'left' : edges[:-1], 'right' : edges[1:]})
    hist_df['ret_interval'] = [(round(left,3), round(right,3)) for left, right in zip(hist_df['left'], hist_df['right'])]
    hist_source.data = hist_df
    similarity_df = nlp_df(id_)
    similarity_source.data = similarity_df
    price_chart.title.text = (id_ + ' price').title() 
    rolling_plot.title.text = (id_ + " correlation vs. bitcoin and ethereum").title()
    histogram.title.text = (id_ + " histogram").title()
    
def update_scatter():
    id_ = coin_id.value
    tag = coin_tag.value
    scatter_df = scatter_plot_df(tag, start = start_date.value, end = end_date.value)
    scatter_source.data = scatter_df
    
'''
These last four functions run along side and are a specific parameter for bokehs 
on_change method. What they do is when the on_change method is called they run,
and switch the widgets value options. Through these functions the update dashboard 
and scatter functions are also ran. You MUST use these functions alongside the on_change
method.


'''

def coin_id_change(attrname, old, new):
    coin_id.completions = coin_ids #changes option choices
    update_dash()
    
def tag_change(attrname, old, new):
    coin_tag.options = coin_tags #changes option choices
    update_scatter()

def date_change_1(attrname, old, new):
    update_dash()
    update_scatter()
    
def date_change_2(attrname, old, new):
    update_dash()
    update_scatter()
    
coin_id.on_change('value', coin_id_change)
coin_tag.on_change('value', tag_change)
start_date.on_change('value', date_change_1)
end_date.on_change('value', date_change_2)


# Creating layout for dashbaord

In [38]:
#Create a layout for the dashboard
col1 = column(coin_id, start_date, end_date, price_chart,rolling_plot)
col2 = column(stats_table, histogram)
col3 = column(coin_tag, scatter_plot)
layout = row(col1,col2, col3, similarity_table)


In [39]:
#create a bokeh server and add our layout/dashboard to it.
curdoc().add_root(layout)
curdoc().title = 'Cryptocurrency Dashboard'

# bokeh serve --show {name of notebook.ipynb}
